# **Imports & Iperparameters**

In [ ]:
import os
import cv2
import tf_keras
import pathlib
import patoolib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import main.models.I_SRCNN.utilities.utils_srcnn as us
import main.models.I_SRCNN.utilities.architectures_srcnn as arcs

from tqdm import tqdm
from glob import glob
from tf_keras.layers import *
from tf_keras.optimizers import *
from tf_keras.utils import plot_model, img_to_array
from tf_keras.preprocessing.image import *
from tf_keras.preprocessing import  image_dataset_from_directory


**Iperparametri** \
Questi sono gli Iperparametri che possiamo decidere noi

In [ ]:
# SuperRes
BATCH_SIZE  = 64
SCALE       = 2.0

# DataSet
INPUT_DIM   = 32
LABEL_SIZE  = [20, 64]
STRIDE      = 14
PAD         = [int((INPUT_DIM - LABEL_SIZE[0]) / 2.0), 0]

# Model Training
EPOCHS = 200

# Random Seed
SEED        = 42
tf.random.set_seed(SEED)

# **Load Data**

Nell'Articolo Scientifico che descrive la tecnica SRCNN: "Image Super-Resolution Using Deep
Convolutional Networks", viene esplicitamente detto che per addestrare la rete si è utilizzato un training set relativamente piccolo.

> "we use a relatively small training set,\
>  that consists of 91 images"

In questa implementazione useremo le stesse 91 immagini proposte nell'Articolo.
Tali Dati sono contenuti in un archivio che quindi dovrà prima essere estratto.

In [ ]:
# Come prima cosa definiamo il percorso base di questo file
current_path = (os.path.dirname(os.path.realpath(os.getcwd())) + "\\" + os.path.basename(os.getcwd()))
current_path

In [ ]:
# Estraiamo i Dati dall'archivio nella cartella ".\data\SRCNN_Dataset_Train"
data_zip_path = current_path + "\images\SRCNN_Dataset_Train.rar"
data_imgs_path = patoolib.extract_archive(data_zip_path, outdir=(current_path + "\\images"))

In [ ]:
# Definiamo una lista contenente i percorsi di ogni immagine
data_imgs_path = (current_path + "\images\SRCNN_Dataset_Train")
file_train_pattern = (pathlib.Path(data_imgs_path) / "*.bmp")
file_train_pattern = str(file_train_pattern)
dataset_train_paths = [*glob(file_train_pattern)]
print("number of images:", len(dataset_train_paths))

**Visualize the Data**

In [ ]:
img = plt.imread(np.random.choice(dataset_train_paths))
plt.imshow(img)
plt.axis("off")
print(img_to_array(img).shape)

# **Create Augmented Data**

Nell'Articolo viene specificato come le 91 Immagini iniziali vengano successivamente trattate attraverso delle operazioni di "crop" e "downscale" che servono per creare delle coppie di sotto-immagini della grandezza desiderata le quali andranno a costituire il vero e proprio Training Set con cui verrà addestrata la Rete

> "In the training phase, the ground truth images {Xi}\
> are prepared as sub-images randomly cropped from the training images..."\
\
> "...thus the 91-image dataset can be decomposed into 24,800 sub-images,\
> which are extracted from original images"

Nel nostro caso useremo due strade diverse per la creazione di queste coppie di sotto-immagini.
Questa differenziazione nasce dalla volontà di creare delle modifiche sulla struttura base dell'architettura SRCNN. Alcune modifiche infatti necessitano degli Iperparametri diversi rispetto ad altre.

In [ ]:
# Definiamo i percorsi delle Immagini che formeranno il Dataset
train_residual_dir_lr = current_path + "\\data\\train_residual\\lr"
train_residual_dir_hr = current_path + "\\data\\train_residual\\hr"
train_vanilla_dir_lr = current_path + "\\data\\train_vanilla\\lr"
train_vanilla_dir_hr = current_path + "\\data\\train_vanilla\\hr"
train_residual_dir_lr, train_residual_dir_hr, train_vanilla_dir_lr, train_vanilla_dir_hr

In [ ]:
# Iperparametri:
# - LABEL_SIZE  = 20
# - PAD         = int((INPUT_DIM - LABEL_SIZE) / 2)

for image_path in tqdm(dataset_train_paths):
  filename = pathlib.Path(image_path).stem
  image = us.load_img(image_path)
  image = us.img_to_array(image)
  image = image.astype(np.uint8)
  image = us.tight_crop_image(image, SCALE)
  scaled = us.downsize_upsize_image(image, SCALE)

  height, width = image.shape[:2]
  for y in range(0, height - INPUT_DIM + 1, STRIDE):
    for x in range(0, width - INPUT_DIM + 1, STRIDE):
      input  = us.crop_input(scaled, x, y, INPUT_DIM)
      output = us.crop_output(image, x, y, PAD=PAD[0], LABEL_SIZE=LABEL_SIZE[0])

      input_path = (f"\\{filename}_{x}_{y}_input.bmp")
      output_path = (f"\\{filename}_{x}_{y}_output.bmp")

      cv2.imwrite(train_vanilla_dir_lr + input_path, cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
      cv2.imwrite(train_vanilla_dir_hr + output_path, cv2.cvtColor(output, cv2.COLOR_BGR2RGB))

In [ ]:
# Iperparametri:
# - LABEL_SIZE  = 64
# - PAD         = 0

for image_path in tqdm(dataset_train_paths):
  filename = pathlib.Path(image_path).stem
  image = us.load_img(image_path)
  image = us.img_to_array(image)
  image = image.astype(np.uint8)
  image = us.tight_crop_image(image, SCALE)

  height, width = image.shape[:2]
  for y in range(0, height - INPUT_DIM + 1, STRIDE):
    for x in range(0, width - INPUT_DIM + 1, STRIDE):
      output = us.crop_output(image, x, y, PAD=PAD[1], LABEL_SIZE=LABEL_SIZE[1])
      input  = us.resize_image(output, 1.0/SCALE)
      
      input_path = (f"\\{filename}_{x}_{y}_input.bmp")
      output_path = (f"\\{filename}_{x}_{y}_output.bmp")

      cv2.imwrite(train_residual_dir_lr + input_path, cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
      cv2.imwrite(train_residual_dir_hr + output_path, cv2.cvtColor(output, cv2.COLOR_BGR2RGB))

# **Create Dataset**

In [ ]:
# Scegli quali coppie di immagini formeranno il Dataset
train_dir_lr = [train_vanilla_dir_lr, train_residual_dir_lr]
train_dir_hr = [train_vanilla_dir_hr, train_residual_dir_hr]
images_to_use = 0

In [ ]:
# Train Set
train_data_lr = image_dataset_from_directory(
    directory=train_dir_lr[images_to_use],
    labels=None,
    label_mode=None,
    image_size=(INPUT_DIM, INPUT_DIM),
    interpolation="bilinear",
    batch_size=BATCH_SIZE,
    seed=42
)

train_data_hr = image_dataset_from_directory(
    directory=train_dir_hr[images_to_use],
    labels=None,
    label_mode=None,
    image_size=(LABEL_SIZE[0], LABEL_SIZE[0]),
    interpolation="bilinear",
    batch_size=BATCH_SIZE,
    seed=42
)

ds_train = tf.data.Dataset.zip(train_data_lr, train_data_hr)

**Visualize the Data**

In [ ]:
# Define the iterators
i_train = ds_train.as_numpy_iterator();

In [ ]:
# Plot Train Sample
sample_train = i_train.next()
us.plot_images([tf.cast(sample_train[1][0], tf.uint8), tf.cast(sample_train[0][0], tf.uint8)]) # HR, LR

# **Preprocess Data**

Qui mettiamo tutte le operazioni di perprocessamento dei Dati. Quando vorremo addestrare la rete potremo decidere quale di queste effettuare e quale no.

In [ ]:
# Normalize
ds_train = ds_train.map(us.normalizer)

# **Model**

**1. Create the Model**

In [ ]:
# Scegliamo quale Modello vogliamo Addestrare
# (questa scelta dipende anche dal tipo di Dataset che abbiamo creato in precedenza)
input_dim = (INPUT_DIM, INPUT_DIM, 3)
model = arcs.SRCNN(input_dim=input_dim)

**2. Compile the Model**

In [ ]:
# Scegliamo quale Loss e quale Optimizer assegnare al Modello

my_loss = tf_keras.losses.mean_squared_error;
my_opt = tf_keras.optimizers.Adam(learning_rate=0.0003);

model.compile(
  loss=my_loss,
  optimizer=my_opt,
  metrics=[us.PSNR_metric, us.SSIM_metric]
)

In [ ]:
# Eseguiamo il Plot del Modello
model.summary()
plot_model(model, show_shapes=True, rankdir="LR")

**3. Fit the Model**

In [ ]:
# Define TensorBoard Callbacks
tb_callback = tf_keras.callbacks.TensorBoard(log_dir=("logs/" + str(model._name)), histogram_freq=1)

In [ ]:
# Addestriamo il Modello sul Dataset
history = model.fit(ds_train, epochs=EPOCHS, callbacks=[tb_callback])

# **References**

[1] Super-Resolution Convolutional Neural Network for Image Restoration - "https://medium.datadriveninvestor.com/using-the-super-resolution-convolutional-neural-network-for-image-restoration-ff1e8420d846"
[2] SRCNN-keras - "https://github.com/MaokeAI/SRCNN-keras/tree/master"